In [1]:
! pip install pygame
!pip install mutagen
!pip install ttkthemes
import os                             #For directory related operations
import idna                           #For deployment
import sys                            #for using system libraries
import threading                      #For multi-processing
import time                           #For telling current time
import tkinter.messagebox             #For alert boxes
from tkinter import *                 #For Importing all files of tkinter
from tkinter import filedialog        #For texting in alert boxes
from mutagen.mp3 import MP3           #For mp3 operations
from pygame import mixer              #For mp3 operations
from ttkthemes import themed_tk as tk #For set Themes
from tkinter import ttk               #ttk = themed tkinter
import webbrowser

#Creating themed tkinter window and saving it in root variable
root = tk.ThemedTk() 


# Get a list of all themes that can be set
root.get_themes()

#Setting theme named radiance
root.set_theme("radiance")

#Creating status bar SUNKEN 3-D style and text written in west bottom
statusbar = Label(root, text="Welcome to Melody", relief=SUNKEN, anchor=W)
statusbar.pack(side=BOTTOM, fill=X) 

# Create the menubar
menubar = Menu(root)
root.config(menu=menubar)

# Create the submenu
subMenu = Menu(menubar, tearoff=0)

# full path+filename
playlist = []


# Fullpath and filename is required to play the music fron playlist

def browse_file():
    global filename_path
    filename_path = filedialog.askopenfilename()
    add_to_playlist(filename_path) #passing value to function


def add_to_playlist(filename):
    global index
    filename = os.path.basename(filename)
    index=0
    playlistbox.insert(index, filename ) # playlistbox contain filename
    playlist.insert(index, filename_path)
    index += 1
    
def play_next_song():
    global _songs
    _songs = playlist[1:] + [playlist[0]] # move current song to the back of the list 
    pygame.mixer.music.load(_songs[0])
    pygame.mixer.music.play()


'''def play_all():
    stop_music()
    time.sleep(1) #pause for 1 sec
    selected_song = playlistbox.curselection()
    selected_song = int(selected_song[0])
    play_it = playlist[selected_song]
    mixer.music.load(play_it)
    mixer.music.play()
    for i in index :
        selected_song = playlistbox.curselection()
        selected_song = int(selected_song[i])
        play_it = playlist[selected_song]
        mixer.music.load(play_it)
        mixer.music.play()
        statusbar['text'] = "Playing music" + ' - ' + os.path.basename(play_it)
        show_details(play_it)
'''
def callback():
    webbrowser.open_new(r"https://www.jiosaavn.com/")

def callback_lyrics():
    webbrowser.open_new(r"https://genius.com/")

menubar.add_cascade(label="File", menu=subMenu)
#menubar.add_cascade(label="Search",command=callback)

menubar.add_cascade(label="Lyrics", command=callback_lyrics)

subMenu.add_command(label="Open", command=browse_file)
#subMenu.add_command(label="Exit", command=root.destroy)

# Initialize the mixer
mixer.init() 

root.title("Melody")
root.iconbitmap(r'images/melody.ico')

# Root Window -> StatusBar, LeftFrame, RightFrame


#Creating  leftFrame ->Playlist area
leftframe = Frame(root)
leftframe.pack(side=LEFT, padx=30, pady=30)

playlistbox = Listbox(leftframe)
playlistbox.pack()

#creating add btn for adding song in playlist
addBtn = ttk.Button(leftframe, text="+ Add", command=browse_file)
addBtn.pack(side=LEFT)
addBtn.config( width = 5)

searchPhoto = PhotoImage(file='images/search.png')
searchBtn = ttk.Button(leftframe, image=searchPhoto, command=callback)
searchBtn.pack(side=LEFT)
searchBtn.config( width = 5)

#Function for delete song
def del_song():
    selected_song = playlistbox.curselection()
    selected_song = int(selected_song[0])
    playlistbox.delete(selected_song)
    playlist.pop(selected_song)

#Delete btn for deleting song
delBtn = ttk.Button(leftframe, text="- Del", command=del_song)
delBtn.pack(side=LEFT)
delBtn.config( width = 5)

'''
playAllBtn = ttk.Button(leftframe, text="- PlayAll", command=play_all)
playAllBtn.pack(side=LEFT)
'''




# RightFrame -> TopFrame,MiddleFrame and the BottomFrame
rightframe = Frame(root)
rightframe.pack(pady=30)

#Top frame in right frame
topframe = Frame(rightframe)
topframe.pack()


#Label for total length
lengthlabel = Label(topframe, text='Total Length : --:--')
lengthlabel.pack(pady=5)

#Label for current time
currenttimelabel = Label(topframe, text='Current Time : --:--', relief=GROOVE )
currenttimelabel.pack()

#Showing total time length of song of type .mp3 and other
def show_details(play_song):
    file_data = os.path.splitext(play_song)

    if file_data[1] == '.mp3':
        audio = MP3(play_song)
        total_length = audio.info.length
    else:
        a = mixer.Sound(play_song)
        total_length = a.get_length()

    # div - total_length/60...
    mins, secs = divmod(total_length, 60)
    mins = round(mins)
    secs = round(secs)
    timeformat = '{:02d}:{:02d}'.format(mins, secs)
    lengthlabel['text'] = "Total Length" + ' - ' + timeformat

    #start_count function will run for 300 because here t value is 300 
    #so we use threading to do other operations at same time
    t1 = threading.Thread(target=start_count, args=(total_length,))
    t1.start()

#count the time which song has been played..or tells at what time song is
def start_count(t):
    global paused
    # mixer.music.get_busy()-> Returns FALSE when we press the stop button (music stop playing)
    # Continue -> Ignores all of the statements below it. We check if music is paused or not.
    current_time = 0
    while current_time <= t and mixer.music.get_busy():
        if paused:
            continue
        else:
            mins, secs = divmod(current_time, 60)
            mins = round(mins)
            secs = round(secs)
            timeformat = '{:02d}:{:02d}'.format(mins, secs)
            currenttimelabel['text'] = "Current Time" + ' - ' + timeformat
            time.sleep(1)
            current_time += 1

#Function for playing music by playlist or browsing and for resume paused music
def play_music():
    global paused

    if paused:
        mixer.music.unpause()
        statusbar['text'] = "Music Resumed"
        paused = FALSE
    else:
        try:
            stop_music()
            time.sleep(1) #pause for 1 sec
            selected_song = playlistbox.curselection()
            selected_song = int(selected_song[0])
            play_it = playlist[selected_song]
            mixer.music.load(play_it)
            mixer.music.play()
            statusbar['text'] = "Playing music" + ' - ' + os.path.basename(play_it)
            show_details(play_it)
        except:
            tkinter.messagebox.showerror('File not found', 'Melody could not find the file. Please check again.')

#to stop current playing music
def stop_music():
    mixer.music.stop()
    statusbar['text'] = "Music Stopped"

    
paused = FALSE


def pause_music():
    global paused
    paused = TRUE
    mixer.music.pause()
    statusbar['text'] = "Music Paused"


def rewind_music():
    play_music()
    statusbar['text'] = "Music Rewinded"


def set_vol(val):
    volume = float(val) / 100
    mixer.music.set_volume(volume)
    # set_volume of mixer takes value only from 0 to 1. Example - 0, 0.1,0.55,0.54.0.99,1

#Function to mute music
muted = FALSE


def mute_music():
    global muted
    if muted:  # Unmute the music
        mixer.music.set_volume(0.7)
        volumeBtn.configure(image=volumePhoto)
        scale.set(70)
        muted = FALSE
    else:  # mute the music
        mixer.music.set_volume(0)
        volumeBtn.configure(image=mutePhoto)
        scale.set(0)
        muted = TRUE

def repeat_music():
    stop_music()
    time.sleep(1) #pause for 1 sec
    selected_song = playlistbox.curselection()
    selected_song = int(selected_song[0])
    play_it = playlist[selected_song]
    mixer.music.load(play_it)
    mixer.music.play(-1)
    statusbar['text'] = "Repeating Song" + ' - ' + os.path.basename(play_it)
    show_details(play_it)
    
    
#Middle frame in rightframe
middleframe = Frame(rightframe)
middleframe.pack(pady=30, padx=30)

stopPhoto = PhotoImage(file='images/stop.png')
stopBtn = ttk.Button(middleframe, image=stopPhoto, command=stop_music)
stopBtn.grid(row=0, column=0, padx=10)

playPhoto = PhotoImage(file='images/play.png')
playBtn = ttk.Button(middleframe, image=playPhoto, command=play_music)
playBtn.grid(row=0, column=1, padx=10)

pausePhoto = PhotoImage(file='images/pause.png')
pauseBtn = ttk.Button(middleframe, image=pausePhoto, command=pause_music)
pauseBtn.grid(row=0, column=2, padx=10)

# Bottom Frame for volume, rewind, mute etc.

bottomframe = Frame(rightframe)
bottomframe.pack()

repeatPhoto = PhotoImage(file='images/repeat.png')
repeatBtn = ttk.Button(bottomframe, image=repeatPhoto, command=repeat_music)
repeatBtn.grid(row=0, column=1)


rewindPhoto = PhotoImage(file='images/rewind.png')
rewindBtn = ttk.Button(bottomframe, image=rewindPhoto, command=rewind_music)
rewindBtn.grid(row=0, column=0)

mutePhoto = PhotoImage(file='images/mute.png')
volumePhoto = PhotoImage(file='images/volume.png')
volumeBtn = ttk.Button(bottomframe, image=volumePhoto, command=mute_music)
volumeBtn.grid(row=0, column=2)

scale = ttk.Scale(bottomframe, from_=0, to=100, orient=HORIZONTAL, command=set_vol)
scale.set(70)  # implement the default value of scale when music player starts
mixer.music.set_volume(0.7)
scale.grid(row=0, column=3, pady=45, padx=30)

def on_closing():
    stop_music()
    root.destroy()


root.protocol("WM_DELETE_WINDOW", on_closing)
root.mainloop()

You are using pip version 9.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 9.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 9.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


pygame 1.9.5
Hello from the pygame community. https://www.pygame.org/contribute.html
